## **Ex2: Logistic Regression Consulting Project**

Binary Customer Churn
A marketing agency has many customers that use their service to produce ads for the
client/customer websites. They've noticed that they have quite a bit of churn in clients. They
basically randomly assign account managers right now, but want you to create a machine learning
model that will help predict which customers will churn (stop buying their service) so that they can
correctly assign the customers most at risk to churn an account manager. Luckily they have some
historical data, can you help them out? Create a classification algorithm that will help classify
whether or not a customer churned. Then the company can test this against incoming data for
future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:
Name : Name of the latest contact at Company
- Age: Customer Age
- Total_Purchase: Total Ads Purchased
- Account_Manager: Binary 0=No manager, 1= Account manager assigned
- Years: Totaly Years as a customer
- Num_sites: Number of websites that use the service.
- Onboard_date: Date that the name of the latest contact was onboarded
- Location: Client HQ Address
- Company: Name of Client Company

Once you've created the model and evaluated it, test out the model on some new data (you can
think of this almost like a hold-out set) that your client has provided, saved under
new_customers.csv. The client wants to know which customers are most likely to churn given this
data (they don't have the label yet).

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
!tar xf spark-3.1.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql import functions as F

In [5]:
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_7'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_7


In [7]:
from pyspark import SparkContext
sc = SparkContext()


In [9]:
spark = SparkSession(sc)

In [11]:
data = spark.read.csv('data/customer_churn.csv',inferSchema=True, header=True)
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [12]:
data.describe().show()


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [13]:
data.columns


['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites'],outputCol='features')

In [16]:
output = assembler.transform(data)


In [17]:
final_data = output.select('features','churn')


### **Train_Test_Split**

In [18]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [20]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')

In [21]:
fitted_churn_model = lr_churn.fit(train_churn)
training_sum = fitted_churn_model.summary


In [22]:
training_sum.predictions.describe().show()


+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               613|                613|
|   mean|0.1566068515497553|0.12071778140293637|
| stddev|0.3637264967840307|0.32606508276167345|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### **Evaluate results**

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)


In [25]:
pred_and_labels.predictions.show()


+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.75285969059638...|[0.99144679900008...|       0.0|
|[25.0,9672.03,0.0...|    0|[5.17411312615370...|[0.99437063367850...|       0.0|
|[26.0,8939.61,0.0...|    0|[7.00901728267257...|[0.99909711969940...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.71994622381205...|[0.99673083521455...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.51244567269150...|[0.99979908861228...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.51955371846990...|[0.82047275396802...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.28075185654507...|[0.99493715933423...|       0.0|
|[29.0,10203.18,1....|    0|[3.89294998117148...|[0.98002212978581...|       0.0|
|[29.0,11274.46,1....|    0|[4.63468108890195...|[0.99038415923200...|       0.0|
|[29.0,12711.15,

In [26]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [27]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.7371244635193134

### **Predict on brand new unlabeled data**

In [28]:
final_lr_model = lr_churn.fit(final_data)


In [30]:
new_customers = spark.read.csv('data/new_customers.csv',inferSchema=True,
                              header=True)

In [31]:
new_customers.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [32]:
test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company','prediction').show()


+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



### **The client wants to know which customers are most likely to churn given this data (they don't have the label yet).**

In [34]:
'''
They are Cannon-Benson,Barron-Robertson Sexton-GOlden, and Parks-Robbins!
'''

'\nThey are Cannon-Benson,Barron-Robertson Sexton-GOlden, and Parks-Robbins!\n'